# Importar librerías

In [1]:
import pandas as pd
import numpy as np

# Utilidades

In [2]:
def compliance(x, position):
        
    status_list = x.values.tolist()
    
    if position=='auditor':
        
        aux_list = [i for i in status_list if i == 'C']
        percentage = len(aux_list)/len(status_list)
        
    if position=='boss':
        
        aux_list = [i for i in status_list if i == 'R']
        percentage = len(aux_list)/len(status_list)
        
    if position=='subg':
        
        counter = status_list.count('A')
        percentage = counter/len(status_list)
               
    return percentage
            

def status(x, position):

    for i in x:
                   
        if position=='auditor':

            if (i=='Aprobada') or (i=='Completada'):
                result = 'C'
            else:
                result = 'P'

        if position=='boss':

            if (i == 'Aprobada'):
                result = 'R'
            else:
                result = 'P'

        if position=='subg':
            result = status_subg(x)
        
    return result


def status_subg(x):
    
    values_list = x.unique()
    
    if (len(values_list)==1) and (values_list[0]=='Aprobada'):
        result = 'A'
    
    else:
        result = 'P'
    
    return result


# Carga de archivos

In [3]:
input_path = 'C:/Users/ISINIESCARC/OneDrive - Asociación Chilena de Seguridad/Documentos/pamc/supervision/'
input_path_af = 'C:/Users/ISINIESCARC/OneDrive - Asociación Chilena de Seguridad/Documentos/pamc/supervision/archivos_fijos/'
output_path = 'C:/Users/ISINIESCARC/OneDrive - Asociación Chilena de Seguridad/Documentos/pamc/bases/current_data/'

xls0 = pd.ExcelFile(output_path + 'PAMC_Hitos.xlsx')                   # Archivo descargado con RPA
xls1 = pd.ExcelFile(input_path + 'PAMC_Auditorías.xlsx')              # Archivo descargado con RPA
xls2 = pd.ExcelFile(input_path + 'PAMC_Plan_de_Trabajo.xlsx')         # Archivo descargado con RPA
xls3 = pd.ExcelFile(input_path + 'PAMC_Supervisión_Etapas.xlsx')      # Archivo descargado con RPA
xls4 = pd.ExcelFile(input_path_af + 'pdt_map.xlsx')                      # Archivo base, no cambia
xls5 = pd.ExcelFile(input_path_af + 'etapas_map.xlsx')                   # Archivo base, no cambia

fechas = pd.read_excel(xls0, 'Sheet1')
auditorias = pd.read_excel(xls1, 'Sheet1')
pdt = pd.read_excel(xls2, 'Sheet1')
etapas = pd.read_excel(xls3, 'Sheet1')

pdt_map = pd.read_excel(xls4, 'Hoja1')
etapas_map = pd.read_excel(xls5, 'Hoja1')

# Limpieza

#### Obtener fechas para filtrar el periodo correspondiente

In [4]:
start_date, end_date = '2021-01-01', '2021-03-31'
fechas = fechas[['Auditoría', 'Auditoría Periodo Actual', 'Hito', 'Fecha Real']]
fechas = fechas[fechas['Hito']=='Emisión Informe']
fechas['Fecha Real'] = fechas['Fecha Real'].astype(str).str[:10]
fechas = fechas[(fechas['Fecha Real']>=start_date)&(fechas['Fecha Real']<=end_date)]

#### Limpieza de Planes de trabajo y etapas: Normalización de nombres

In [5]:
# Eliminar columna sin nombre
pdt_map = pdt_map.dropna(subset=['Unnamed: 1'])
etapas_map = etapas_map.dropna(subset=['Unnamed: 1'])

# Renombrar columna con valores normalizados
pdt_map.rename(columns={'Unnamed: 1': 'plan_norm'}, inplace=True)
etapas_map.rename(columns={'Unnamed: 1': 'etapa_norm'}, inplace=True)

#### Limpieza de Planes de trabajo y etapas: Crear DataFrame normalizado

In [6]:
pdt_norm = pdt.merge(pdt_map, left_on='Plan de Trabajo', right_on='Plan de Trabajo', how='inner')[['Auditoría', 'Auditoría Periodo Actual', 'plan_norm', 'Estado de Finalización']].rename(columns={'Estado de Finalización': 'estado_pdt'})
etapas_norm = etapas.merge(etapas_map, left_on='Etapa', right_on='Etapa', how='inner')[['Auditoría', 'Auditoría Periodo Actual', 'Plan de Trabajo', 'etapa_norm', 'Estado de Finalización']].merge(pdt_map, on='Plan de Trabajo', how='inner').rename(columns={'Estado de Finalización': 'estado_etapa'})

# Ejecución

#### Juntar planes de trabajo, etapas y estado de auditorías en un DataFrame

In [7]:
key_columns = ['Auditoría', 'Auditoría Periodo Actual','plan_norm']
data = pdt_norm.merge(etapas_norm, on=key_columns, how='left').reset_index(drop=True).merge(auditorias, on='Auditoría', how='left')

#### Ordenar columnas

In [8]:
columns = ['Auditoría', 'Tipo de Auditoría', 'Auditoría Periodo Actual', 'plan_norm', 'estado_pdt', 'etapa_norm', 'estado_etapa', 'Estado de Auditoría / Aprobación de Auditoría']
data = data[columns]

#### Filtrar periodo correspondiente

In [9]:
columns.append('Fecha Real')
data = data.merge(fechas, on=['Auditoría', 'Auditoría Periodo Actual'], how='inner')[columns]
data = data.rename(columns={'Estado de Auditoría / Aprobación de Auditoría':'estado_aud'})

#### Llenar valores nulos

Dado que existen auditorías sin etapas, la llenamos con lo mismo de plan de trabajo

In [10]:
data['etapa_norm'] = data['etapa_norm'].fillna(data['plan_norm'])
data['estado_etapa'] = data['estado_etapa'].fillna(data['estado_pdt'])

In [11]:
data

,Auditoría,Tipo de Auditoría,Auditoría Periodo Actual,plan_norm,estado_pdt,etapa_norm,estado_etapa,estado_aud,Fecha Real
0,GAI_IE_001_2021 - Posible Agresión HT (pabellón),NaN,2021,1. PEP - Planificación,Aprobada,1. PEP - Planificación,Aprobada,NaN,2021-01-14
1,GAI_IE_001_2021 - Posible Agresión HT (pabellón),NaN,2021,2. PEE - Ejecución,Aprobada,2. PEE - Ejecución,Aprobada,NaN,2021-01-14
2,GAI_IE_001_2021 - Posible Agresión HT (pabellón),NaN,2021,3. PEC - Comunicación,Aprobada,3. PEC - Comunicación,Aprobada,NaN,2021-01-14
3,GAI_003_2021 - Auditoria Continua 4to Trimestr...,NaN,2021,1. PEP - Planificación,Aprobada,1. PEP - Planificación,Aprobada,NaN,2021-02-02
4,GAI_003_2021 - Auditoria Continua 4to Trimestr...,NaN,2021,2. PEE - Ejecución,Aprobada,2. PEE - Ejecución,Aprobada,NaN,2021-02-02
...,...,...,...,...,...,...,...,...,...
90,GAI_2021_ACHS - Seguimientos Auditoría Externa...,NaN,2021,3. PEC - Comunicación,Aprobada,3. PEC - Comunicación,Aprobada,NaN,2021-02-01
91,GAI_2021_CEM - Seguimientos Auditoría Externa ...,NaN,2021,3. PEC - Comunicación,Aprobada,3. PEC - Comunicación,Aprobada,NaN,2021-02-01
92,GAI_2021_ESACHS - Seguimientos Auditoría Exter...,NaN,2021,3. PEC - Comunicación,Aprobada,3. PEC - Comunicación,Aprobada,NaN,2021-02-01
93,GAI_2021_ESACHS Transporte - Seguimientos Audi...,NaN,2021,3. PEC - Comunicación,Aprobada,3. PEC - Comunicación,Aprobada,NaN,2021-02-01


#### Agregar analizar estado de acuerdo a parámetros PAMC (C, R, A, P)

 - C: Completado, aplica para la revisión a nivel de auditor
 - R: Revisado, aplica para la revisión a nivel de jefe
 - A: Aprobado, aplica para la revisión a nivel de subgerente
 - P: Pendiente, aplica para todos los niveles

In [12]:
# El auditor hizo su trabajo cuando la ETAPA está aprobada o completada
status_auditor = data.groupby(['Auditoría', 'Auditoría Periodo Actual', 'plan_norm', 'etapa_norm']).apply(lambda x: status(x['estado_etapa'], position='auditor')).reset_index().rename(columns={0: 'status_auditor'})

In [13]:
status_auditor

,Auditoría,Auditoría Periodo Actual,plan_norm,etapa_norm,status_auditor
0,2021-04 - RED - Agencia Puerto Montt-Coyhaique,2021,1. PEP - Planificación,PEP-01 - Inicio y Programa General de Auditoría,C
1,2021-04 - RED - Agencia Puerto Montt-Coyhaique,2021,1. PEP - Planificación,PEP-06 - Diseño de Programa de Trabajo,C
2,2021-04 - RED - Agencia Puerto Montt-Coyhaique,2021,2. PEE - Ejecución,PEE-07 - Ejecutar el Programa Específico de Au...,C
3,2021-04 - RED - Agencia Puerto Montt-Coyhaique,2021,3. PEC - Comunicación,PEC-11 - Reunión de Cierre con Ejecutivos,C
4,2021-04 - RED - Agencia Puerto Montt-Coyhaique,2021,3. PEC - Comunicación,PEC-12 - Emisión Informe y Cierre Administrativo,C
...,...,...,...,...,...
90,GAI_xxx - Validación de los controles asociado...,2021,2. PEE - Ejecución,PEE-07 - Ejecutar el Programa Específico de Au...,P
91,GAI_xxx - Validación de los controles asociado...,2021,2. PEE - Ejecución,PEE-09 - Presentación y revisión de los result...,P
92,GAI_xxx - Validación de los controles asociado...,2021,3. PEC - Comunicación,PEC-10 - Redactar y Calificar Observaciones,P
93,GAI_xxx - Validación de los controles asociado...,2021,3. PEC - Comunicación,PEC-11 - Reunión de Cierre con Ejecutivos,P


In [14]:
# El jefe hizo su trabajo cuando la ETAPA está aprobada
status_boss = data.groupby(['Auditoría', 'Auditoría Periodo Actual', 'plan_norm', 'etapa_norm']).apply(lambda x: status(x['estado_etapa'], position='boss')).reset_index().rename(columns={0: 'status_boss'})

In [15]:
status_boss

,Auditoría,Auditoría Periodo Actual,plan_norm,etapa_norm,status_boss
0,2021-04 - RED - Agencia Puerto Montt-Coyhaique,2021,1. PEP - Planificación,PEP-01 - Inicio y Programa General de Auditoría,R
1,2021-04 - RED - Agencia Puerto Montt-Coyhaique,2021,1. PEP - Planificación,PEP-06 - Diseño de Programa de Trabajo,R
2,2021-04 - RED - Agencia Puerto Montt-Coyhaique,2021,2. PEE - Ejecución,PEE-07 - Ejecutar el Programa Específico de Au...,R
3,2021-04 - RED - Agencia Puerto Montt-Coyhaique,2021,3. PEC - Comunicación,PEC-11 - Reunión de Cierre con Ejecutivos,R
4,2021-04 - RED - Agencia Puerto Montt-Coyhaique,2021,3. PEC - Comunicación,PEC-12 - Emisión Informe y Cierre Administrativo,R
...,...,...,...,...,...
90,GAI_xxx - Validación de los controles asociado...,2021,2. PEE - Ejecución,PEE-07 - Ejecutar el Programa Específico de Au...,P
91,GAI_xxx - Validación de los controles asociado...,2021,2. PEE - Ejecución,PEE-09 - Presentación y revisión de los result...,P
92,GAI_xxx - Validación de los controles asociado...,2021,3. PEC - Comunicación,PEC-10 - Redactar y Calificar Observaciones,P
93,GAI_xxx - Validación de los controles asociado...,2021,3. PEC - Comunicación,PEC-11 - Reunión de Cierre con Ejecutivos,P


In [16]:
# El subgerente hizo su trabajo cuando el PLAN DE TRABAJO está aprobad0
status_subg = data.groupby(['Auditoría', 'Auditoría Periodo Actual', 'plan_norm']).apply(lambda x: status(x['estado_pdt'], position='subg')).reset_index().rename(columns={0: 'status_subg'})

In [17]:
status_subg

,Auditoría,Auditoría Periodo Actual,plan_norm,status_subg
0,2021-04 - RED - Agencia Puerto Montt-Coyhaique,2021,1. PEP - Planificación,A
1,2021-04 - RED - Agencia Puerto Montt-Coyhaique,2021,2. PEE - Ejecución,A
2,2021-04 - RED - Agencia Puerto Montt-Coyhaique,2021,3. PEC - Comunicación,A
3,2021-05 - RED - Agencia Elqui,2021,1. PEP - Planificación,A
4,2021-05 - RED - Agencia Elqui,2021,2. PEE - Ejecución,A
5,2021-05 - RED - Agencia Elqui,2021,3. PEC - Comunicación,A
6,2021-06 - RED - Agencia Ruta del Sol,2021,1. PEP - Planificación,A
7,2021-06 - RED - Agencia Ruta del Sol,2021,2. PEE - Ejecución,A
8,2021-06 - RED - Agencia Ruta del Sol,2021,3. PEC - Comunicación,A
9,2021-07 - RED - Agencia Osorno -Valdivia,2021,1. PEP - Planificación,A


In [18]:
# Se une horizontalmente los resultados anteriores (auditor, jefe y subgerente)
full_data = status_auditor.merge(status_boss, on=['Auditoría', 'Auditoría Periodo Actual', 'plan_norm', 'etapa_norm'], how='inner').merge(status_subg, on=['Auditoría', 'Auditoría Periodo Actual', 'plan_norm'], how='left')

In [19]:
full_data.tail(10)

,Auditoría,Auditoría Periodo Actual,plan_norm,etapa_norm,status_auditor,status_boss,status_subg
85,GAI_xxx - Validación de los controles asociado...,2021,1. PEP - Planificación,PEP-02 - Investigación Preliminar,P,P,P
86,GAI_xxx - Validación de los controles asociado...,2021,1. PEP - Planificación,PEP-03 - Definir y Documentar Plan de Auditori...,P,P,P
87,GAI_xxx - Validación de los controles asociado...,2021,1. PEP - Planificación,PEP-04 - Presentación Plan de Trabajo a Ejecut...,P,P,P
88,GAI_xxx - Validación de los controles asociado...,2021,1. PEP - Planificación,PEP-05 - Identificación / Evaluación de Riesgo...,P,P,P
89,GAI_xxx - Validación de los controles asociado...,2021,1. PEP - Planificación,PEP-06 - Diseño de Programa de Trabajo,P,P,P
90,GAI_xxx - Validación de los controles asociado...,2021,2. PEE - Ejecución,PEE-07 - Ejecutar el Programa Específico de Au...,P,P,P
91,GAI_xxx - Validación de los controles asociado...,2021,2. PEE - Ejecución,PEE-09 - Presentación y revisión de los result...,P,P,P
92,GAI_xxx - Validación de los controles asociado...,2021,3. PEC - Comunicación,PEC-10 - Redactar y Calificar Observaciones,P,P,P
93,GAI_xxx - Validación de los controles asociado...,2021,3. PEC - Comunicación,PEC-11 - Reunión de Cierre con Ejecutivos,P,P,P
94,GAI_xxx - Validación de los controles asociado...,2021,3. PEC - Comunicación,PEC-12 - Emisión Informe y Cierre Administrativo,P,P,P


#### Cálculo del porcentaje de cumplimiento

In [20]:
# Se calcula el porcentaje de Completado, Revisado o Aprobado (según el cargo) sobre el total de etapas por auditoría
result_auditor = status_auditor.groupby(['Auditoría', 'Auditoría Periodo Actual']).apply(lambda x: compliance(x['status_auditor'], position='auditor')).reset_index().rename(columns={0: 'percentage_auditor'})
result_boss = status_boss.groupby(['Auditoría', 'Auditoría Periodo Actual']).apply(lambda x: compliance(x['status_boss'], position='boss')).reset_index().rename(columns={0: 'percentage_boss'})
result_subg = status_subg.groupby(['Auditoría', 'Auditoría Periodo Actual']).apply(lambda x: compliance(x['status_subg'], position='subg')).reset_index().rename(columns={0: 'percentage_subg'})

#Se consolida la información en un DataFrame
result_per_audit = result_auditor.merge(result_boss, on=['Auditoría', 'Auditoría Periodo Actual'], how='inner').merge(result_subg, on=['Auditoría', 'Auditoría Periodo Actual'], how='left')

#### Resumen del porcentaje de cumplimiento

In [21]:
n_auditor_real = len(status_auditor[status_auditor['status_auditor']=='C'])
n_boss_real = len(status_boss[status_boss['status_boss']=='R'])
n_subg_real = len(status_subg[status_subg['status_subg']=='A'])

compliance_auditor = n_auditor_real/len(status_auditor)
compliance_boss = n_boss_real/len(status_boss)
compliance_subg = n_subg_real/len(status_subg)

summary_results_dict = {
    'Percentage': [compliance_auditor, compliance_boss, compliance_subg],
    'N_Real': [n_auditor_real, n_boss_real, n_subg_real],
    'N_Total': [len(status_auditor), len(status_boss), len(status_subg)],
    'Cargos': ['Auditor', 'Jefe', 'Subgerente']
}

summary_results = pd.DataFrame(summary_results_dict).set_index('Cargos')

In [22]:
list(result_per_audit['Auditoría'].unique())

['2021-04 - RED - Agencia Puerto Montt-Coyhaique',
 '2021-05 - RED - Agencia Elqui',
 '2021-06 - RED - Agencia Ruta del Sol',
 '2021-07 - RED - Agencia Osorno -Valdivia',
 'AE_2021_RCR - Seguimientos Auditoría Externa PwC 2020',
 'GAI_002_2021 - Clínica Los Coihues',
 'GAI_003_2021 - Auditoria Continua 4to Trimestre 2020',
 'GAI_005_2021 - Revisión Carpetas de Colaboradores HCS (Concepción)',
 'GAI_006_2021 - Arqueo Administración de cajas HCS (Concepción)',
 'GAI_007_2021 - Control de efectivo y Dctos. Valorados ACHS',
 'GAI_04_2021 (RCR) - Auditoría Continua RCR 2 semestre',
 'GAI_2021 - Atención Clínica Ambulatoria',
 'GAI_2021_ACHS - Seguimientos Auditoría Externa PwC 2020',
 'GAI_2021_CEM - Seguimientos Auditoría Externa PwC 2020',
 'GAI_2021_ESACHS - Seguimientos Auditoría Externa PwC 2020',
 'GAI_2021_ESACHS Transporte - Seguimientos Auditoría Externa PwC 2020',
 'GAI_IE_001_2021 - Posible Agresión HT (pabellón)',
 'GAI_IE_002_2021 - Posible Acoso Laboral/Ex colaborador ACHS (ex

In [23]:
display(full_data)
display(result_per_audit)
display(summary_results)

,Auditoría,Auditoría Periodo Actual,plan_norm,etapa_norm,status_auditor,status_boss,status_subg
0,2021-04 - RED - Agencia Puerto Montt-Coyhaique,2021,1. PEP - Planificación,PEP-01 - Inicio y Programa General de Auditoría,C,R,A
1,2021-04 - RED - Agencia Puerto Montt-Coyhaique,2021,1. PEP - Planificación,PEP-06 - Diseño de Programa de Trabajo,C,R,A
2,2021-04 - RED - Agencia Puerto Montt-Coyhaique,2021,2. PEE - Ejecución,PEE-07 - Ejecutar el Programa Específico de Au...,C,R,A
3,2021-04 - RED - Agencia Puerto Montt-Coyhaique,2021,3. PEC - Comunicación,PEC-11 - Reunión de Cierre con Ejecutivos,C,R,A
4,2021-04 - RED - Agencia Puerto Montt-Coyhaique,2021,3. PEC - Comunicación,PEC-12 - Emisión Informe y Cierre Administrativo,C,R,A
...,...,...,...,...,...,...,...
90,GAI_xxx - Validación de los controles asociado...,2021,2. PEE - Ejecución,PEE-07 - Ejecutar el Programa Específico de Au...,P,P,P
91,GAI_xxx - Validación de los controles asociado...,2021,2. PEE - Ejecución,PEE-09 - Presentación y revisión de los result...,P,P,P
92,GAI_xxx - Validación de los controles asociado...,2021,3. PEC - Comunicación,PEC-10 - Redactar y Calificar Observaciones,P,P,P
93,GAI_xxx - Validación de los controles asociado...,2021,3. PEC - Comunicación,PEC-11 - Reunión de Cierre con Ejecutivos,P,P,P


,Auditoría,Auditoría Periodo Actual,percentage_auditor,percentage_boss,percentage_subg
0,2021-04 - RED - Agencia Puerto Montt-Coyhaique,2021,1.000000,1.000000,1.0
1,2021-05 - RED - Agencia Elqui,2021,1.000000,1.000000,1.0
2,2021-06 - RED - Agencia Ruta del Sol,2021,1.000000,1.000000,1.0
3,2021-07 - RED - Agencia Osorno -Valdivia,2021,1.000000,1.000000,1.0
4,AE_2021_RCR - Seguimientos Auditoría Externa P...,2021,1.000000,1.000000,1.0
5,GAI_002_2021 - Clínica Los Coihues,2021,1.000000,1.000000,1.0
6,GAI_003_2021 - Auditoria Continua 4to Trimestr...,2021,1.000000,1.000000,1.0
7,GAI_005_2021 - Revisión Carpetas de Colaborado...,2021,1.000000,1.000000,1.0
8,GAI_006_2021 - Arqueo Administración de cajas ...,2021,1.000000,1.000000,1.0
9,GAI_007_2021 - Control de efectivo y Dctos. Va...,2021,0.000000,0.000000,0.0


,Percentage,N_Real,N_Total
Cargos,,,
Auditor,0.736842,70,95
Jefe,0.736842,70,95
Subgerente,0.820000,41,50


In [24]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(output_path + 'pamc1_supervision.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
summary_results.to_excel(writer, sheet_name='resumen')
result_per_audit.to_excel(writer, sheet_name='resultados_por_auditoria')
full_data.to_excel(writer, sheet_name='datos')

# Close the Pandas Excel writer and output the Excel file.
writer.save()